In [2]:
import os
import numpy as np
import pandas as pd
os.environ["NAMESPACE"]="research"
os.environ["PROFILE"]="local"
from agrobrain_util.runtime.evironment import RuntimeEnv
from agrobrain_util.infra.app_config import application_config as cfg

import shutil
import glob
from skimage import io
import matplotlib.pyplot as plt
import json
import random
import glob
import datetime
from tqdm import tqdm
from datetime import datetime

from PIL import Image


env = RuntimeEnv()
categories_dict = cfg['tags']['categories']

DATA_DIR = "data"
IMAGE_DIR = "images"


def get_wide_images_list(current_df):
    l  = []
    for i in tqdm(range(0, len(current_df['imageID']), 10)):
        # print(current_df['imageID'][i])
        wide_image_id = env.eti_api.get_matching_wide_images(list(current_df['imageID'][i:i+10]))
        # print(wide_image_id)
        l.append(wide_image_id)
    wide_images_list = [item for sublist in l for item in sublist]
    return wide_images_list

def create_cat_dict(cat_list):
    cat_dict_types = {}
    cat_dict_names = {}
    for l in cat_list:
        cat_dict_types[l['id']] = l['type']
        if 'name' in l:
            cat_dict_names[l['id']] = l['name']
        else:
            cat_dict_names[l['id']] = 'NoName'
    cat_dict_types[0] = None
    cat_dict_names[0] = None
    return cat_dict_types, cat_dict_names

categories_hierarchy = env.eti_api.get_categories_hierarchy()
cat_dict_types, cat_dict_names = create_cat_dict(categories_hierarchy[9]['subCategories'])

could not find stamp in environment variables
could not find stamp in environment variables
could not find KUBEFLOW_API_BASIC_AUTH_TOKEN in environment variables
could not find AI_RABBITMQ_CONNECTION_STRING in environment variables
could not find AI_MONGO_DB_CONNECTION_STRING in environment variables
could not find TARANIS_DB_CONNECTION_STRING in environment variables
could not find GLOBAL_DB_CONNECTION_STRING in environment variables
could not find AI_MONGO_DB_CONNECTION_STRING_NEW in environment variables


# Read and create datasets

In [ ]:
# IMAGES DF - ALL THE IMAGES THAT CONTAINS WEEDS

weeds_images_df = pd.read_csv(os.path.join(DATA_DIR, "weeds_images_df_light.csv"))

weeds_images_df['weed_types_ids_list'] = weeds_images_df['weed_types_ids_list'].apply(eval)
weeds_images_df['uploadDate_time'] = weeds_images_df['uploadDate'].apply(datetime.fromtimestamp)
weeds_images_df['uploadDate_date'] = weeds_images_df['uploadDate_time'].apply(lambda x: x.to_pydatetime().date())
filtered_weeds_images_df = weeds_images_df[(weeds_images_df['cameraAngle'] > -95) & (weeds_images_df['cameraAngle'] < -85)]
images_df = filtered_weeds_images_df
weeds_images_df = 0
filtered_weeds_images_df = 0

In [ ]:
# CREATE "TOP WEEDS DF" - FILTER DF AND TAKE THE MOST PROBABLE IMAGES (TO CONTAIN WEEDS) FROM EACH CROP 


NUM_OF_IMAGES_FROM_EACH_CROP = 200

top_weeds_corn = images_df[images_df['cropName']=='Corn'].nlargest(NUM_OF_IMAGES_FROM_EACH_CROP, 'num_weed_tags')
top_weeds_soy = images_df[images_df['cropName']=='Soybean'].nlargest(NUM_OF_IMAGES_FROM_EACH_CROP, 'num_weed_tags')
top_weeds_df = pd.concat([top_weeds_corn, top_weeds_soy], axis='rows', ignore_index=True)

# top_weeds_df['wideImageID'] = env.eti_api.get_matching_wide_images(list(top_weeds_df['imageID']))
# top_weeds_df['wideImageID'] = top_weeds_df['wideImageID'].fillna(-1).astype(int)

top_weeds_df.to_csv(os.path.join(DATA_DIR, f"top_weeds_df_to_each_crop_type_{NUM_OF_IMAGES_FROM_EACH_CROP}_images_from_each.csv"))


In [ ]:
images_df.columns

In [ ]:
# CREATE "SAMPLED DATAFRAME" BY UNIFORM SAMPLING THE DISTRIBUTION OF "NUM WEED TAGS" - MANUALLY AND INFERENCE

def get_sampled_df(current_df, name='', n=20):
    weights_manual_tags = current_df['weed_manualTagsCount'] / current_df['weed_manualTagsCount'].sum()
    weights_inference_tags = current_df['weed_inferenceTagsCount'] / current_df['weed_inferenceTagsCount'].sum()

    sampled_df_manual = current_df.sample(n=n, weights=weights_manual_tags, replace=False).reset_index()
    sampled_df_inference = current_df.sample(n=n, weights=weights_inference_tags, replace=False).reset_index()

    sampled_df = pd.concat([sampled_df_manual, sampled_df_inference])
    sampled_df['wideImageID'] = get_wide_images_list(sampled_df)

    fig, axes = plt.subplots(nrows=1, ncols=2)
    axes[0].hist(sampled_df['num_weed_tags'], bins = 200, color='c')
    axes[0].set_title(f"sampled {name} dataframe num_weed_tags")

    axes[1].hist(current_df['num_weed_tags'], bins = 200, color='c')
    axes[1].set_title(f"full {name} images dataframe num_weed_tags")

    fig.set_size_inches(10, 5)
    plt.tight_layout()
    plt.show()
    return sampled_df


N = 100

# # images_df['weed_types_ids'] = images_df['weed_types_ids_list'].apply(lambda x: x[0])
# images_df['weed_inferenceTagsCount'] = images_df['weed_inferenceTagsCount'].apply(eval)
# images_df['weed_inferenceTagsCount'] = images_df['weed_inferenceTagsCount'].apply(lambda x: x[0])
images_df['weed_manualTagsCount'] = images_df['num_weed_tags'] - images_df['weed_inferenceTagsCount']


corn_df = images_df[images_df['cropName']=='Corn'].reset_index()
corn_sampled_df = get_sampled_df(corn_df, name='corn', n=N)

soy_df = images_df[images_df['cropName']=='Soybean'].reset_index()
soy_sampled_df = get_sampled_df(soy_df, name='soy', n=N)

sampled_df = pd.concat([corn_sampled_df, soy_sampled_df])

sampled_df_csv_path = os.path.join(DATA_DIR, f"sampled_df_by_crop_types_and_weight_sample_manual_and_inference_tag_count_{N}_from_each.csv")
sampled_df.to_csv(sampled_df_csv_path)

In [ ]:
interesting_weed_df = pd.read_csv(os.path.join(DATA_DIR, "interesting_weed_df.csv"))
images_to_drop_from_interesting_weed_df = [8980673, 8980669, 8980681, 8980665]
interesting_weed_df = interesting_weed_df[~interesting_weed_df['imageID'].isin(images_to_drop_from_interesting_weed_df)]

small_num_of_tags_df_soy = pd.read_csv(os.path.join(DATA_DIR, "small_num_of_tags_df_soy.csv"))
small_num_of_tags_df_corn = pd.read_csv(os.path.join(DATA_DIR, "small_num_of_tags_df_corn.csv"))
soy_with_volunteer_corn_df = pd.read_csv(os.path.join(DATA_DIR, "soy_with_volunteer_corn_df_sorted.csv"))
corn_with_volunteer_corn_df = pd.read_csv(os.path.join(DATA_DIR, "corn_with_volunteer_corn_df_sorted.csv"))
sampled_df = pd.read_csv(os.path.join(DATA_DIR, f"sampled_df_by_crop_types_and_weight_sample_manual_and_inference_tag_count_100_from_each.csv"))
top_weeds_df = pd.read_csv(os.path.join(DATA_DIR, "top_weeds_df_to_each_crop_type_200_images_from_each_with_wide.csv"))

len(interesting_weed_df), len(small_num_of_tags_df_soy), len(small_num_of_tags_df_corn), len(soy_with_volunteer_corn_df), len(corn_with_volunteer_corn_df), len(sampled_df), len(top_weeds_df)

In [ ]:
# SAMPLE FROM THE SMALLER DATASETS AND SAVE

small_num_of_tags_df_soy = pd.read_csv(os.path.join(DATA_DIR, "small_num_of_tags_df_soy.csv")).sample(50).reset_index(drop=True)
small_num_of_tags_df_corn = pd.read_csv(os.path.join(DATA_DIR, "small_num_of_tags_df_corn.csv")).sample(50).reset_index(drop=True)
soy_with_volunteer_corn_df = pd.read_csv(os.path.join(DATA_DIR, "soy_with_volunteer_corn_df_sorted.csv")).sample(50).reset_index(drop=True)
corn_with_volunteer_corn_df = pd.read_csv(os.path.join(DATA_DIR, "corn_with_volunteer_corn_df_sorted.csv")).sample(50).reset_index(drop=True)
sampled_df = pd.read_csv(os.path.join(DATA_DIR, f"sampled_df_by_crop_types_and_weight_sample_manual_and_inference_tag_count_100_from_each.csv"))
top_weeds_df = pd.read_csv(os.path.join(DATA_DIR, "top_weeds_df_to_each_crop_type_200_images_from_each_with_wide.csv"))

combined_df = pd.concat([top_weeds_df, sampled_df, soy_with_volunteer_corn_df, corn_with_volunteer_corn_df, small_num_of_tags_df_soy, small_num_of_tags_df_corn], ignore_index=True)
combined_df.to_csv(os.path.join(DATA_DIR, "1000_images_dataframe_for_tagging_methodology_task_2023_07_18.csv"))

In [ ]:
# SHOW ZOOM-WIDE COUPLE OF IMAGES FROM THE DATAFRAME

# df_to_show = images_df.sample(10).reset_index(drop=True)
df_to_show = small_num_of_tags_df_soy.sample(10).reset_index(drop=True)
# df_to_show['wideImageID'] = get_wide_images_list(df_to_show)



for i in range(len(df_to_show)):
    example_image_id = df_to_show['imageID'][i]
    # SHOW IMAGES
    print(f"Zoom image id: {example_image_id}")
    image_data = df_to_show[df_to_show['imageID'] == example_image_id].reset_index(drop=True)

    # image_data = df_to_show[df_to_show['imageID'] == example_image_id]
    matching_wide_image_id = int(image_data.at[0, 'wideImageID'])
    print(f"Wide image id: {matching_wide_image_id}")
    image_num_tags = image_data.at[0, 'num_weed_tags']

    image_crop_name = image_data.at[0, 'cropName']
    weed_types = image_data.at[0, 'weed_types_ids_list']
    # weed_types = eval(image_data.at[0, 'weed_types_ids_list'])
    weed_types = list(map(cat_dict_names.get, weed_types))


    im_path = env.download_image(int(example_image_id))
    image = io.imread(im_path)

    wide_im_path = env.download_image(int(matching_wide_image_id))
    wide_image = io.imread(wide_im_path)

    fig, axes = plt.subplots(nrows=1, ncols=2)
    axes[0].imshow(image, extent=[0, image.shape[1], 0, image.shape[0]])
    axes[1].imshow(wide_image, extent=[0, wide_image.shape[1], 0, wide_image.shape[0]])


    for ax in axes:
        ax.set_xticks([])
        ax.set_yticks([])
    fig.set_size_inches(10, 5)
    plt.suptitle(f"Crop Type: {image_crop_name}\nZoom Image ID: {example_image_id}, Wide Image ID: {matching_wide_image_id}\nWeed types: {weed_types}")
    plt.tight_layout()
    plt.show()


# Download 1000 images to local machine 

In [19]:
combined_df = pd.read_csv(os.path.join(DATA_DIR, "1000_images_dataframe_for_tagging_methodology_task_2023_07_18.csv"))
combined_df = combined_df.dropna(subset=['wideImageID'])
wide_images_ids_list = list(combined_df['wideImageID'])

In [20]:
# SAVE IMAGES IDS LIST TO OTHER USES
infestation_1000_image_ids_json_file_path = os.path.join(DATA_DIR, "infestation_1000_image_ids.json")
with open(infestation_1000_image_ids_json_file_path, "w") as json_file:
    json.dump(wide_images_ids_list, json_file)

In [ ]:
local_wide_images_path = combined_df['wideImageID'].astype(int).apply(env.download_image)

In [ ]:
# MOVE THE IMAGES TO FOLDER 
destination_folder = os.path.join(IMAGE_DIR, "1000_images_tagging_methodology_task_2023_07_18")
os.makedirs(destination_folder, exist_ok=True)
for image_path in local_wide_images_path:
    image_name = os.path.basename(image_path)
    destination_path = os.path.join(destination_folder, image_name)
    try:
        shutil.move(image_path, destination_path)
    except:
        print(f"problem with {image_path}")